### SQLAlchemy - Introduction

Based on the SQLAlchemy <a href="https://docs.sqlalchemy.org/en/14/tutorial/">documentation</a>.

SQLAlchemy is presented as two distinct APIs, one building on top of the other. These APIs are known as **Core** and **ORM**:

- **SQLAlchemy Core** is the foundational architecture for SQLAlchemy as a *database toolkit*. The library provides tools for managing connectivity to a database, interacting with database queries and results, and programmatic construction of SQL statements.

- **SQLAlchemy ORM** builds upon the Core to provide optional *object relational mapping* capabilities. The ORM provides an additional configuration layer allowing user-defined Python classes to be mapped to database tables and other constructs, as well as an object persistence mechanism known as the `Session`. It then extends the Core-level SQL Expression Language to allow SQL queries to be composed and invoked in terms of user-defined objects.

In [ ]:
# Uncomment for installing the SQL Alchemy library.
#!pip install sqlalchemy

In [ ]:
# importing the library
import sqlalchemy
# checking the version - just for demonstration
sqlalchemy.__version__ 

'1.4.20'

### Establishing connectivity - the `Engine`

---
Any SQLAlchemy application shoudl start with an object called `Engine`. This object acts as a central source of connections to a particular database, providing both a factory as well as a holding space called `connection pool` for these database connections. 

The `engine` is typically a global object created just once for a particular database server, and is configured using a URL string which will describe how it should connect to the database host or backend.

For this tutorial, we will use an *in-memory-only& SQLite database. This is an easy way to test things without needing to have an actual pre-existing database set up. The Engine is created by using `create_engine()` and passing a string URL with following parameters:

- `sqlite`: what kind of database will be used? In this case, the `sqlite` portion above links in SQLAlchemy to an object known as the `dialect`.

- `pysqlite`: what DBAPI are we using? The Python DBAPI is a third party driver that SQLAlchemy uses to interact with a particular database. In this case, we’re using `pysqlite`, which represents the `sqlite3` standard library interface for SQLite.

- `/:memory:`: how do we locate the database? In this case, our URL includes the phrase `/:memory:`, which indicates that we will be using an in-memory-only database.

We also set `echo=True`, which will instruct the Engine to log all of the SQL commands it emits to a Python logger that will write to standard out; and `create_engine.future=True`, so that we make full use of 2.0 style usage.


In [ ]:
from sqlalchemy import create_engine
# creating the engine object
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

### Working with transactions and the DBAPI
---
The `Engine` object has two interactive endpoints that can be used for executing transactions: `Connection` and `Result`. When using the ORM, the `Engine` is managed by a <a href="https://docs.sqlalchemy.org/en/14/glossary.html#term-facade">facade</a> object called `Session`, which emphasizes a transactional and SQL execution pattern that is largely identical to that of the `Connection`.

### Getting the connection
---

The `Engine` object provides a unit of connectivity to the database called the `Connection`. When working with the Core directly, the `Connection` object is how all interaction with the database is done. As the Connection represents an open resource against the database, we want to always limit the scope of our use of this object to a specific context, and the best way to do that is by using Python context manager form, also known as the `with` statement. Below we illustrate “Hello World”, using a textual SQL statement. Textual SQL is emitted using a construct called `text()`.



In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
  result = conn.execute(text("select 'hello world'"))
  print(result.all())

2021-08-13 13:21:21,861 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:21,864 INFO sqlalchemy.engine.Engine select 'hello world'
2021-08-13 13:21:21,870 INFO sqlalchemy.engine.Engine [generated in 0.00893s] ()
[('hello world',)]
2021-08-13 13:21:21,872 INFO sqlalchemy.engine.Engine ROLLBACK


The default behavior of the Python DBAPI is that a transaction is always in progress; when the scope of the connection is released, a `ROLLBACK` is emitted to end the transaction. The transaction is not committed automatically; when we want to commit data we normally need to call `Connection.commit()`.

**IMPORTANT:** The result of `SELECT` is returned in an object called `Result`, that should be consumed within the *connect* block, i.e. should not be passed along outside of the scope of the connection.

### Committing changes
---
There are two ways of commiting changes to the database when you need to store some data:

- `Connection.commit()`: this is known as **commit as you go**. We should emit this command **inside** the block in which we acquired a connection.


In [ ]:
# "commit as you go"
# we are creating a table and inserting some data, then commiting the changes to the database.
with engine.connect() as conn:
  conn.execute(text("CREATE TABLE some_table (x int, y int)"))
  conn.execute(
      text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
      [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
      )
  conn.commit()

2021-08-13 13:21:21,890 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:21,891 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2021-08-13 13:21:21,893 INFO sqlalchemy.engine.Engine [generated in 0.00362s] ()
2021-08-13 13:21:21,896 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2021-08-13 13:21:21,898 INFO sqlalchemy.engine.Engine [generated in 0.00256s] ((1, 1), (2, 4))
2021-08-13 13:21:21,901 INFO sqlalchemy.engine.Engine COMMIT


- `Engine.begin()`: the *connect* block is defined as a *transaction block* up front. For this mode of operation, we use the `Engine.begin()` method to acquire the connection, rather than the `Engine.connect()` method. This method will both manage the scope of the Connection and also enclose everything inside of a transaction with `COMMIT`` at the end, assuming a successful block, or `ROLLBACK` in case of exception raise. This style may be referred towards as **begin once**.

In [ ]:
# "begin once"
# the connection block is implicitly treated as a transaction
with engine.begin() as conn:
  conn.execute(
      text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
      [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
  )

2021-08-13 13:21:21,921 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:21,924 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2021-08-13 13:21:21,926 INFO sqlalchemy.engine.Engine [cached since 0.02972s ago] ((6, 8), (9, 10))
2021-08-13 13:21:21,928 INFO sqlalchemy.engine.Engine COMMIT


### Fetching rows
---

We’ll first illustrate the `Result` object more closely by making use of the rows we’ve inserted previously, running a textual `SELECT` statement on the table we’ve created.

In [ ]:
with engine.connect() as conn:
  result = conn.execute(text("SELECT x, y FROM some_table"))
  for row in result:
    print(f"x: {row.x}  y: {row.y}")

2021-08-13 13:21:21,953 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:21,955 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2021-08-13 13:21:21,957 INFO sqlalchemy.engine.Engine [generated in 0.00454s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2021-08-13 13:21:21,960 INFO sqlalchemy.engine.Engine ROLLBACK


Noticed that the `Result` object returned by the `SELECT` command is an iterable Python object of result rows (i.e., it implements the iterator interface so that we can iterate over the collection of Row objects directly.

`Result` has lots of methods for fetching and transforming rows, such as the `Result.all()` method, which returns a list of all Row objects. 

The `Row objects` themselves are intended to act like Python `named tuples`, so we have a variety of ways to access them:

- **Tuple Assignment:** this is the most Python-idiomatic style, which is to assign variables to each row positionally as they are received.

- **Integer Index:** tuples are Python sequences, so regular integer access is available too.

- **Attribute Name:** as these are Python named tuples, the tuples have dynamic attribute names matching the names of each column. These names are normally the names that the SQL statement assigns to the columns in each row. 

- **Mapping Access:** to receive rows as Python mapping objects, which is essentially a read-only version of Python’s interface to the common `dict` object, the `Result` may be transformed into a `MappingResult` object using the `Result.mappings()` modifier; this is a result object that yields dictionary-like `RowMapping` objects rather than `Row` objects.

In [ ]:
# tuple assignment
with engine.connect() as conn:
  result = conn.execute(text("select x, y from some_table"))
  
  for x, y in result.fetchall():
    # illustrate use with Python f-strings
    print(f"x: {x}  y: {y}")

2021-08-13 13:21:21,985 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:21,987 INFO sqlalchemy.engine.Engine select x, y from some_table
2021-08-13 13:21:21,988 INFO sqlalchemy.engine.Engine [generated in 0.00281s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2021-08-13 13:21:21,992 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
# integer index
with engine.connect() as conn:
  result = conn.execute(text("select x, y from some_table"))
  
  for row in result:
    x = row[0]
    print(f"x: {x}")

2021-08-13 13:21:22,016 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:22,020 INFO sqlalchemy.engine.Engine select x, y from some_table
2021-08-13 13:21:22,022 INFO sqlalchemy.engine.Engine [cached since 0.03698s ago] ()
x: 1
x: 2
x: 6
x: 9
2021-08-13 13:21:22,024 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
# attribute name
with engine.connect() as conn:
  result = conn.execute(text("select x, y from some_table"))
  
  for row in result:
    y = row.y
    print(f"Row: {row.x} {row.y}")

2021-08-13 13:21:22,035 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:22,042 INFO sqlalchemy.engine.Engine select x, y from some_table
2021-08-13 13:21:22,043 INFO sqlalchemy.engine.Engine [cached since 0.05772s ago] ()
Row: 1 1
Row: 2 4
Row: 6 8
Row: 9 10
2021-08-13 13:21:22,045 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
# mapping access
with engine.connect() as conn:
  result = conn.execute(text("select x, y from some_table"))
  
  for dict_row in result.mappings():
    x = dict_row['x']
    y = dict_row['y']
    print(f"x: {x}  y: {y}")

2021-08-13 13:21:22,056 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:22,059 INFO sqlalchemy.engine.Engine select x, y from some_table
2021-08-13 13:21:22,060 INFO sqlalchemy.engine.Engine [cached since 0.07467s ago] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2021-08-13 13:21:22,062 INFO sqlalchemy.engine.Engine ROLLBACK


### Parameterized queries

The `Connection.execute()` method also accepts parameters, which are referred towards as <a href="https://docs.sqlalchemy.org/en/14/glossary.html#term-bound-parameters">bound parameters</a>. A rudimentary example might be if we wanted to limit our `SELECT` statement only to rows that meet a certain criteria, such as rows where the *y* value were greater than a certain value that is passed in to a function.

In order to achieve this such that the SQL statement can remain fixed and that the driver can properly sanitize the value, we add a `WHERE` criteria to our statement that names a new parameter called *y*; the `text()` construct accepts these using a colon format `:y`. The actual value for `:y` is then passed as the second argument to `Connection.execute()` in the form of a dictionary.


In [ ]:
# example of parameterized query having only one parameter
with engine.connect() as conn:
  result = conn.execute(
      text("SELECT x, y FROM some_table WHERE y > :y"),
      {"y": 2} # dict containing the parameter to be passed to the query
  )
  
  for row in result:
    print(f"x: {row.x}  y: {row.y}")

2021-08-13 13:21:22,073 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:22,076 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2021-08-13 13:21:22,079 INFO sqlalchemy.engine.Engine [generated in 0.00635s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2021-08-13 13:21:22,081 INFO sqlalchemy.engine.Engine ROLLBACK


**Multiple parameters**

We can send multi parameters to the `Connection.execute()` method by passing a list of dictionaries instead of a single dictionary, thus allowing the single SQL statement to be invoked against each parameter set individually.

Behind the scenes, the `Connection` object uses a DBAPI feature known as <a href="https://www.python.org/dev/peps/pep-0249/#id18">cursor.executemany()</a>. This method performs the equivalent operation of invoking the given SQL statement against each parameter set individually. The DBAPI may optimize this operation in a variety of ways, by using prepared statements, or by concatenating the parameter sets into a single SQL statement in some cases.




In [ ]:
# example of multi-parameterized query
with engine.connect() as conn:
  conn.execute(
      text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
      [{"x": 11, "y": 12}, {"x": 13, "y": 14}]  # list of dict to be iterated over during the query 
  )
  conn.commit()

2021-08-13 13:21:22,094 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:22,099 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2021-08-13 13:21:22,100 INFO sqlalchemy.engine.Engine [cached since 0.2042s ago] ((11, 12), (13, 14))
2021-08-13 13:21:22,101 INFO sqlalchemy.engine.Engine COMMIT


**Bundling parameters with a statement**

Another example of using **single** parameterized queries is through the `TextClause.bindparams()` method; this is a <a href="https://docs.sqlalchemy.org/en/14/glossary.html#term-generative">generative method</a> that returns a new copy of the SQL construct with additional state added, in this case the parameter values we want to pass along.






In [ ]:
# example of single parameterized query using the bindparams() construct
stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)

with engine.connect() as conn:
  result = conn.execute(stmt) # the SQL statement to be executed
  for row in result:
    print(f"x: {row.x}  y: {row.y}")

2021-08-13 13:21:22,113 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:22,115 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2021-08-13 13:21:22,117 INFO sqlalchemy.engine.Engine [generated in 0.00449s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2021-08-13 13:21:22,118 INFO sqlalchemy.engine.Engine ROLLBACK


### Executing with an ORM Session
---
The fundamental transactional / database interactive object when using the ORM is called `Session`. In SQLAlchemy, this object is used in a manner very similar to that of the `Connection`, and in fact as the `Session` is used, it refers to a `Connection` internally which it uses to emit SQL. When the `Session` is used with non-ORM constructs, it behaves very similarly to the `Connection`.

The `Session` has a few different creational patterns, but here we will illustrate the most basic one that tracks exactly with how the `Connection` is used which is to construct it within a context manager.

Compare the next command with the previous one and notice how we changed from `engine.connect() as conn` to `with Session(engine) as session`, and then make use of the `Session.execute()` method just like we do with the `Connection.execute()`.

In [ ]:
# importing the necessaty library / objects
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
# replacing Connection by Session
with Session(engine) as session:
  result = session.execute(stmt)
  for row in result:
    print(f"x: {row.x}  y: {row.y}")

2021-08-13 13:21:22,232 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:22,234 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2021-08-13 13:21:22,238 INFO sqlalchemy.engine.Engine [cached since 0.1252s ago] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2021-08-13 13:21:22,241 INFO sqlalchemy.engine.Engine ROLLBACK


Below, we invoked an `UPDATE` statement using the bound-parameter, “executemany” style of execution introduced earlier, ending the block with a “commit as you go” `Session.commit()` method.

In [ ]:
with Session(engine) as session:
  result = session.execute(
      text("UPDATE some_table SET y=:y WHERE x=:x"),
      [{"x": 9, "y":11}, {"x": 13, "y": 15}]
  )
  session.commit()

2021-08-13 13:21:22,256 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-13 13:21:22,265 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2021-08-13 13:21:22,268 INFO sqlalchemy.engine.Engine [generated in 0.00242s] ((11, 9), (15, 13))
2021-08-13 13:21:22,272 INFO sqlalchemy.engine.Engine COMMIT
